<a href="https://colab.research.google.com/github/manankapoor23/DSAI_Fine_Tuning/blob/main/01_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
!pip install beautifulsoup4 requests tqdm langdetect

import os
os.makedirs("data/raw", exist_ok=True)


In [8]:
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import re

def extract_article_text(url):
    try:
        r = requests.get(url, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")

        for tag in soup(["script", "style", "nav", "footer", "header"]):
            tag.decompose()

        text = soup.get_text(separator=" ")
        text = re.sub(r"\s+", " ", text)
        return text.strip()
    except:
        return None

def collect_news(site_url, limit=200):
    r = requests.get(site_url, timeout=10)
    soup = BeautifulSoup(r.text, "html.parser")

    links = set()
    for a in soup.find_all("a", href=True):
        href = a["href"]
        if href.startswith("http"):
            links.add(href)

    articles = []
    for link in tqdm(list(links)[:limit], desc=f"Scraping {site_url}"):
        text = extract_article_text(link)
        if text and len(text) > 800:
            articles.append(text)

    return articles
news_sources = [
    "https://jagbani.punjabkesari.in/",
    "https://www.indiapress.org/gen/news.php/Daily_Ajit/400x60/0"
]

news_data = []

for site in news_sources:
    news_data.extend(collect_news(site, limit=250))

print("Collected news articles:", len(news_data))


Scraping https://jagbani.punjabkesari.in/: 100%|██████████| 243/243 [03:37<00:00,  1.12it/s]
Scraping https://www.indiapress.org/gen/news.php/Daily_Ajit/400x60/0: 0it [00:00, ?it/s]

Collected news articles: 165


In [9]:
with open("data/raw/punjabi_news.txt", "w", encoding="utf-8") as f:
    for article in news_data:
        f.write(article + "\n\n")

print("Punjabi news data saved successfully ✅")


Punjabi news data saved successfully ✅


In [10]:
import re
import unicodedata
import os

# Gurmukhi Unicode range
GURMUKHI_REGEX = re.compile(r'[\u0A00-\u0A7F]')

def clean_text(text):
    text = unicodedata.normalize("NFC", text)
    text = re.sub(r'https?://\S+', '', text)
    text = re.sub(r'\S+@\S+', '', text)
    text = re.sub(r'[^A-Za-z\u0A00-\u0A7F\s।!?]', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    return text.strip()

def is_punjabi_gurmukhi(text, threshold=0.7):
    if len(text) < 30:
        return False
    punjabi_chars = len(GURMUKHI_REGEX.findall(text))
    return (punjabi_chars / len(text)) >= threshold



In [11]:
input_file = "data/raw/punjabi_news.txt"

with open(input_file, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Loaded characters:", len(raw_text))


Loaded characters: 7947203


In [12]:
# Split aggressively (news pages are noisy)
chunks = re.split(r'[।!?]', raw_text)

clean_punjabi = []

for chunk in chunks:
    chunk = clean_text(chunk)
    if is_punjabi_gurmukhi(chunk):
        clean_punjabi.append(chunk)

print("Punjabi-only sentences kept:", len(clean_punjabi))


Punjabi-only sentences kept: 3132


In [13]:
for i in range(5):
    print(f"\nSample {i+1}:\n{clean_punjabi[i]}")



Sample 1:
ਪਰ ਜੇ ਤੁਸੀਂ ਗਲਤੀ ਨਾਲ Block ਸਿਲੈਕਟ ਕੀਤਾ ਸੀ ਜਾਂ ਫਿਰ ਭਵਿੱਖ ਚ ਤੁਸੀਂ ਨੋਟਿਫਿਕੇਸ਼ਨ ਪਾਉਣਾ ਚਾਹੁੰਦੇ ਹੋ ਤਾਂ ਥੱਲੇ ਦਿੱਤੇ ਨਿਰਦੇਸ਼ਾਂ ਦਾ ਪਾਲਨ ਕਰੋ

Sample 2:
ਲਰਨਰ ਨੇ ਨੈਕਸਟ ਜੇਨ Facebook Tumblr Linkedin Twitter ਕਿਰਗਿਓਸ ਬ੍ਰਿਸਬੇਨ ਵਿੱਚ ਏਟੀਪੀ ਟੂਰ ਤੇ ਵਾਪਸੀ ਲਈ ਤਿਆਰ ਨਿੱਕ ਕਿਰਗਿਓਸ ਅਗਲੇ ਮਹੀਨੇ ਹੋਣ ਵਾਲੇ ਬ੍ਰਿਸਬੇਨ ਇੰਟਰਨੈਸ਼ਨਲ ਲਈ ਵਾਈਲਡਕਾਰਡ ਐਂਟਰੀ ਪ੍ਰਾਪਤ ਕਰਨ ਤੋਂ ਬਾਅਦ ਮਹੀਨਿਆਂ ਵਿੱਚ ਆਪਣਾ ਪਹਿਲਾ ਏਟੀਪੀ ਟੂਰ ਮੈਚ ਖੇਡਣ ਲਈ ਤਿਆਰ ਹੈ

Sample 3:
ਸੈਮੂਅਲ ਲੋਪੇਜ਼ PM TENNIS ਆਰੀਅਨਾ ਸਬਾਲੇਂਕਾ ਫਿਰ ਡਬਲਯੂ ਟੀ ਏ ਦੀ ਸਾਲ ਦੀ ਸਰਵੋਤਮ ਖਿਡਾਰਣ ਬਣੀ ਆਰੀਅਨਾ ਸਬਾਲੇਂਕਾ ਨੇ ਸੋਮਵਾਰ ਨੂੰ ਲਗਾਤਾਰ ਦੂਜੀ ਵਾਰ ਡਬਲਯੂ ਟੀ ਏ ਦੀ ਸਾਲ ਦੀ ਸਰਵੋਤਮ ਖਿਡਾਰਣ ਦਾ ਇਨਾਮ ਜਿੱਤਿਆ

Sample 4:
ਉਸ ਨੇ ਅਮਰੀਕੀ ਓਪਨ ਜਿੱਤਣ ਹੋਰ ਗ੍ਰੈਂਡ ਸਲੈਮ AM TENNIS ਸਾਲ ਦੀ Top Newcomer ਬਣੀ ਕੈਨੇਡਾ ਦੀ ਸਾਲਾ ਟੈਨਿਸ ਸਟਾਰ ਵਿਕਟੋਰੀਆ ਮਬੋਕੋ ਕੈਨੇਡਾ ਦੀ ਉੱਭਰਦੀ ਟੈਨਿਸ ਖਿਡਾਰਣ ਵਿਕਟੋਰੀਆ ਮਬੋਕੋ ਨੂੰ ਵੂਮਨਜ਼ ਟੈਨਿਸ ਅਸੋਸੀਏਸ਼ਨ ਵੱਲੋਂ ਸਾਲ ਦੀ ਟਾਪ ਨਿਊਕਮਰ ਐਲਾਨਣ ਦਾ ਅਹਿਮ ਫੈਸਲਾ AM TENNIS OnlyFans ਤੇ ਐਂਟਰੀ ਮਾਰਨ ਜਾ ਰਹੀ ਇਹ ਖੂਬਸੂਰਤ ਟੈਨਿਸ ਸਟਾਰ ਫਰਾਂਸ ਦੀ ਪ੍ਰੋਫੈਸ਼ਨਲ ਟੈਨਿਸ ਸਟਾਰ ਓਸੀਅਨ ਡੋਡਿਨ ਹਮੇਸ਼ਾ ਤੋਂ ਵਿਵਾਦਪੂਰਨ ਕੰਮਾਂ ਨੂੰ ਲੈ ਕੇ ਚਰਚਾ ਵਿਚ ਰਹਿੰਦੀ ਹੈ

Sample 5:

In [14]:
os.makedirs("data/processed", exist_ok=True)

output_file = "data/processed/punjabi_clean.txt"

with open(output_file, "w", encoding="utf-8") as f:
    for sent in clean_punjabi:
        f.write(sent + "\n")

print("Saved clean Punjabi-only file ✅")


Saved clean Punjabi-only file ✅


In [15]:
total_chars = sum(len(s) for s in clean_punjabi)
approx_tokens = total_chars // 4   # LLaMA approx

print("Approx tokens:", approx_tokens)


Approx tokens: 189525


In [16]:
input_file = "data/processed/punjabi_clean.txt"

with open(input_file, "r", encoding="utf-8") as f:
    sentences = [line.strip() for line in f if line.strip()]

print("Sentences before deduplication:", len(sentences))


Sentences before deduplication: 3132


In [17]:
from hashlib import md5

def deduplicate(sentences):
    seen = set()
    unique = []

    for sent in sentences:
        h = md5(sent.encode("utf-8")).hexdigest()
        if h not in seen:
            seen.add(h)
            unique.append(sent)

    return unique

deduped_sentences = deduplicate(sentences)

print("Sentences after deduplication:", len(deduped_sentences))
print("Duplicates removed:", len(sentences) - len(deduped_sentences))


Sentences after deduplication: 1446
Duplicates removed: 1686


In [18]:
for i in range(5):
    print(f"\nSample {i+1}:\n{deduped_sentences[i]}")



Sample 1:
ਪਰ ਜੇ ਤੁਸੀਂ ਗਲਤੀ ਨਾਲ Block ਸਿਲੈਕਟ ਕੀਤਾ ਸੀ ਜਾਂ ਫਿਰ ਭਵਿੱਖ ਚ ਤੁਸੀਂ ਨੋਟਿਫਿਕੇਸ਼ਨ ਪਾਉਣਾ ਚਾਹੁੰਦੇ ਹੋ ਤਾਂ ਥੱਲੇ ਦਿੱਤੇ ਨਿਰਦੇਸ਼ਾਂ ਦਾ ਪਾਲਨ ਕਰੋ

Sample 2:
ਲਰਨਰ ਨੇ ਨੈਕਸਟ ਜੇਨ Facebook Tumblr Linkedin Twitter ਕਿਰਗਿਓਸ ਬ੍ਰਿਸਬੇਨ ਵਿੱਚ ਏਟੀਪੀ ਟੂਰ ਤੇ ਵਾਪਸੀ ਲਈ ਤਿਆਰ ਨਿੱਕ ਕਿਰਗਿਓਸ ਅਗਲੇ ਮਹੀਨੇ ਹੋਣ ਵਾਲੇ ਬ੍ਰਿਸਬੇਨ ਇੰਟਰਨੈਸ਼ਨਲ ਲਈ ਵਾਈਲਡਕਾਰਡ ਐਂਟਰੀ ਪ੍ਰਾਪਤ ਕਰਨ ਤੋਂ ਬਾਅਦ ਮਹੀਨਿਆਂ ਵਿੱਚ ਆਪਣਾ ਪਹਿਲਾ ਏਟੀਪੀ ਟੂਰ ਮੈਚ ਖੇਡਣ ਲਈ ਤਿਆਰ ਹੈ

Sample 3:
ਸੈਮੂਅਲ ਲੋਪੇਜ਼ PM TENNIS ਆਰੀਅਨਾ ਸਬਾਲੇਂਕਾ ਫਿਰ ਡਬਲਯੂ ਟੀ ਏ ਦੀ ਸਾਲ ਦੀ ਸਰਵੋਤਮ ਖਿਡਾਰਣ ਬਣੀ ਆਰੀਅਨਾ ਸਬਾਲੇਂਕਾ ਨੇ ਸੋਮਵਾਰ ਨੂੰ ਲਗਾਤਾਰ ਦੂਜੀ ਵਾਰ ਡਬਲਯੂ ਟੀ ਏ ਦੀ ਸਾਲ ਦੀ ਸਰਵੋਤਮ ਖਿਡਾਰਣ ਦਾ ਇਨਾਮ ਜਿੱਤਿਆ

Sample 4:
ਉਸ ਨੇ ਅਮਰੀਕੀ ਓਪਨ ਜਿੱਤਣ ਹੋਰ ਗ੍ਰੈਂਡ ਸਲੈਮ AM TENNIS ਸਾਲ ਦੀ Top Newcomer ਬਣੀ ਕੈਨੇਡਾ ਦੀ ਸਾਲਾ ਟੈਨਿਸ ਸਟਾਰ ਵਿਕਟੋਰੀਆ ਮਬੋਕੋ ਕੈਨੇਡਾ ਦੀ ਉੱਭਰਦੀ ਟੈਨਿਸ ਖਿਡਾਰਣ ਵਿਕਟੋਰੀਆ ਮਬੋਕੋ ਨੂੰ ਵੂਮਨਜ਼ ਟੈਨਿਸ ਅਸੋਸੀਏਸ਼ਨ ਵੱਲੋਂ ਸਾਲ ਦੀ ਟਾਪ ਨਿਊਕਮਰ ਐਲਾਨਣ ਦਾ ਅਹਿਮ ਫੈਸਲਾ AM TENNIS OnlyFans ਤੇ ਐਂਟਰੀ ਮਾਰਨ ਜਾ ਰਹੀ ਇਹ ਖੂਬਸੂਰਤ ਟੈਨਿਸ ਸਟਾਰ ਫਰਾਂਸ ਦੀ ਪ੍ਰੋਫੈਸ਼ਨਲ ਟੈਨਿਸ ਸਟਾਰ ਓਸੀਅਨ ਡੋਡਿਨ ਹਮੇਸ਼ਾ ਤੋਂ ਵਿਵਾਦਪੂਰਨ ਕੰਮਾਂ ਨੂੰ ਲੈ ਕੇ ਚਰਚਾ ਵਿਚ ਰਹਿੰਦੀ ਹੈ

Sample 5:

In [19]:
deduped_file = "data/processed/punjabi_deduped.txt"

with open(deduped_file, "w", encoding="utf-8") as f:
    for sent in deduped_sentences:
        f.write(sent + "\n")

print("Saved deduplicated Punjabi text ✅")


Saved deduplicated Punjabi text ✅


In [20]:
import json
import os

os.makedirs("data/final", exist_ok=True)

jsonl_file = "data/final/punjabi_llama2.jsonl"

with open(jsonl_file, "w", encoding="utf-8") as f:
    for sent in deduped_sentences:
        record = {"text": sent}
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print("Saved JSONL file for LLaMA-2 fine-tuning ✅")


Saved JSONL file for LLaMA-2 fine-tuning ✅


In [21]:
with open(jsonl_file, "r", encoding="utf-8") as f:
    for _ in range(3):
        print(f.readline())


{"text": "ਪਰ ਜੇ ਤੁਸੀਂ ਗਲਤੀ ਨਾਲ Block ਸਿਲੈਕਟ ਕੀਤਾ ਸੀ ਜਾਂ ਫਿਰ ਭਵਿੱਖ ਚ ਤੁਸੀਂ ਨੋਟਿਫਿਕੇਸ਼ਨ ਪਾਉਣਾ ਚਾਹੁੰਦੇ ਹੋ ਤਾਂ ਥੱਲੇ ਦਿੱਤੇ ਨਿਰਦੇਸ਼ਾਂ ਦਾ ਪਾਲਨ ਕਰੋ"}

{"text": "ਲਰਨਰ ਨੇ ਨੈਕਸਟ ਜੇਨ Facebook Tumblr Linkedin Twitter ਕਿਰਗਿਓਸ ਬ੍ਰਿਸਬੇਨ ਵਿੱਚ ਏਟੀਪੀ ਟੂਰ ਤੇ ਵਾਪਸੀ ਲਈ ਤਿਆਰ ਨਿੱਕ ਕਿਰਗਿਓਸ ਅਗਲੇ ਮਹੀਨੇ ਹੋਣ ਵਾਲੇ ਬ੍ਰਿਸਬੇਨ ਇੰਟਰਨੈਸ਼ਨਲ ਲਈ ਵਾਈਲਡਕਾਰਡ ਐਂਟਰੀ ਪ੍ਰਾਪਤ ਕਰਨ ਤੋਂ ਬਾਅਦ ਮਹੀਨਿਆਂ ਵਿੱਚ ਆਪਣਾ ਪਹਿਲਾ ਏਟੀਪੀ ਟੂਰ ਮੈਚ ਖੇਡਣ ਲਈ ਤਿਆਰ ਹੈ"}

{"text": "ਸੈਮੂਅਲ ਲੋਪੇਜ਼ PM TENNIS ਆਰੀਅਨਾ ਸਬਾਲੇਂਕਾ ਫਿਰ ਡਬਲਯੂ ਟੀ ਏ ਦੀ ਸਾਲ ਦੀ ਸਰਵੋਤਮ ਖਿਡਾਰਣ ਬਣੀ ਆਰੀਅਨਾ ਸਬਾਲੇਂਕਾ ਨੇ ਸੋਮਵਾਰ ਨੂੰ ਲਗਾਤਾਰ ਦੂਜੀ ਵਾਰ ਡਬਲਯੂ ਟੀ ਏ ਦੀ ਸਾਲ ਦੀ ਸਰਵੋਤਮ ਖਿਡਾਰਣ ਦਾ ਇਨਾਮ ਜਿੱਤਿਆ"}



In [22]:
total_chars = sum(len(s) for s in deduped_sentences)
approx_tokens = total_chars // 4

print("Approx tokens:", approx_tokens)


Approx tokens: 78055


In [23]:
print("Average sentence length:",
      sum(len(s) for s in deduped_sentences) / len(deduped_sentences))


Average sentence length: 215.92116182572613
